# Utilities

In [1]:
"""
Main BEHAVIOR demo replay entrypoint
"""

import argparse
import datetime
import os
import pprint
import json

import bddl
import h5py
import numpy as np

import igibson
from igibson.activity.activity_base import iGBEHAVIORActivityInstance
from igibson.render.mesh_renderer.mesh_renderer_cpu import MeshRendererSettings
from igibson.render.mesh_renderer.mesh_renderer_vr import VrSettings
from igibson.simulator import Simulator
from igibson.utils.git_utils import project_git_info
from igibson.utils.ig_logging import IGLogReader, IGLogWriter
from igibson.utils.utils import parse_str_config


def print_class_info(obj, indent=0):
    # Get the class of the object
    cls = obj.__class__

    # Print the class name
    print(" " * indent + f"Class: {cls.__name__}")

    # Iterate through the object's attributes
    for attr_name, attr_value in vars(obj).items():
        print(" " * (indent + 2) + f"{attr_name}:")
        
        # If the attribute is an instance of a custom class, recurse
        if hasattr(attr_value, '__dict__'):
            print_class_info(attr_value, indent + 4)
        else:
            print(" " * (indent + 4) + f"{attr_value}")

    # Print information about methods
    methods = [method for method in dir(cls) if callable(getattr(cls, method)) and not method.startswith("__")]
    if methods:
        print(" " * (indent + 2) + "Methods:")
        for method in methods:
            print(" " * (indent + 4) + method)


def verify_determinism(in_log_path, out_log_path):
    is_deterministic = True
    with h5py.File(in_log_path) as original_file, h5py.File(out_log_path) as new_file:
        for obj in original_file["physics_data"]:
            for attribute in original_file["physics_data"][obj]:
                is_close = np.isclose(
                    original_file["physics_data"][obj][attribute], new_file["physics_data"][obj][attribute]
                )
                is_deterministic = is_deterministic and is_close.all()
                if not is_close.all():
                    print(
                        "Mismatch for obj {} with mismatched attribute {} starting at timestep {}".format(
                            obj, attribute, np.where(is_close == False)[0][0]
                        )
                    )
    return bool(is_deterministic)


def safe_replay_demo(*args, **kwargs):
    """Replays a demo, asserting that it was deterministic."""
    demo_statistics = replay_demo(*args, **kwargs)
    assert (
        demo_statistics["deterministic"] == True
    ), "Replay was not deterministic (or was executed with disable_save=True)."


# Define a class to mock argparse's Namespace object
class Args:
    def __init__(self, vr_log_path, vr_replay_log_path=None, frame_save_path=None, disable_save=False, profile=False, mode=None):
        self.vr_log_path = vr_log_path
        self.vr_replay_log_path = vr_replay_log_path
        self.frame_save_path = frame_save_path
        self.disable_save = disable_save
        self.profile = profile
        self.mode = mode

INFO:root:Importing iGibson (igibson module)
INFO:root:Assets path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/assets
INFO:root:Gibson Dataset path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/g_dataset
INFO:root:iG Dataset path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset
INFO:root:3D-FRONT Dataset path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/threedfront_dataset
INFO:root:CubiCasa5K Dataset path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/cubicasa_dataset
INFO:root:iGibson Key path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/igibson.key
INFO:root:Example path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\examples
INFO:root:Example config path: C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\examples\configs


Indef: FullArgSpec(args=['parsed_state', 'scope', 'object_map'], varargs=None, varkw=None, defaults=(None, None), kwonlyargs=[], kwonlydefaults=None, annotations={})
torch is not available, falling back to rendering to memory(instead of tensor)


In [5]:
from igibson.external.pybullet_tools.utils import ContactResult
from igibson.objects.multi_object_wrappers import ObjectGrouper, ObjectMultiplexer
from igibson.robots.behavior_robot import BRBody
from igibson.object_states.touching import Touching
import pybullet as p

def get_flattened_scene_graph(igbhvr_act_inst):
    data = []
    contact_data = []

    robot = igbhvr_act_inst.simulator.robots[0]
    relevant_objects_raw = set(igbhvr_act_inst.object_scope.values())  # | {robot.parts["left_hand"], robot.parts["right_hand"]}

    # Unwrap any multi-object wrappers
    relevant_objects = set()
    for object in relevant_objects_raw:
        if isinstance(object, ObjectMultiplexer):
            # For multiplexers, we just want to look at
            # the currently-selected object
            relevant_objects.add(object.current_selection())
        elif isinstance(object, ObjectGrouper):
            # For groupers, we want to record ALL the children
            relevant_objects.update(object.objects)
        else:
            # Otherwise, just add the object
            relevant_objects.add(object)

    for object in relevant_objects:
        # If we are processing the robot, first add the robot's contact bodies + grasped objects
        if isinstance(object, BRBody):
            # First, get the actual robot object from the body
            robot = object.parent
            
            states_and_body_ids = []
            
            # Start with the actually grasped objects
            if robot.parts["left_hand"].object_in_hand:
                states_and_body_ids.append(("left_hand_grasped", robot.parts["left_hand"].object_in_hand))
            if robot.parts["right_hand"].object_in_hand:
                states_and_body_ids.append(("right_hand_grasped", robot.parts["right_hand"].object_in_hand))
            
            # Then also list the in-contact bodies
            for hand_name in ["left_hand", "right_hand"]:
                hand = robot.parts[hand_name]
                contact_bodies = [ContactResult(*item[:10]) for item in p.getContactPoints(bodyA=hand.get_body_id())]
                for contact_body in contact_bodies:
                    states_and_body_ids.append((f"{hand_name}_contact", contact_body.bodyUniqueIdB))
                    
            # Convert all of the state_to_body_id tuples into actual objects
            contact_data.extend(
                [
                    ((object.name, state_name, igbhvr_act_inst.simulator.scene.objects_by_id[body_id].name), True)
                    for state_name, body_id in states_and_body_ids
                    if body_id in igbhvr_act_inst.simulator.scene.objects_by_id
                ]
            )
            
        
        # for state_name, state in object.states.items():
        #     try:
        #         # if this is a unary state and the value is a boolean
        #         if state.get_value() in [True, False]:
        #             data.append(((object.name, state_name), state.get_value()))
        #     except:
        #         try:
        #             # if this is a binary state and the value is a boolean (you can check its state with itself is a boolean)
        #             if state.get_value(object) in [True, False]:
        #                 for target_object in relevant_objects:
        #                     data.append(((object.name, state_name, target_object.name), state.get_value(target_object)))
                            # if target_object.name == 'left_hand_1':
                            #         contact_data.append(((object.name, state_name, target_object.name), state.get_value(target_object)))
                            # if target_object.name == 'right_hand_1':
                            #         contact_data.append(((object.name, state_name, target_object.name), state.get_value(target_object)))
                            # if v.name == 'left_hand_1' and key is Touching:
                            #         contact_data.append(((v.name, key, w.name), value.get_value(w)))
                            # if v.name == 'right_hand_1' and key is Touching:
                            #         contact_data.append(((v.name, key, w.name), value.get_value(w)))
                                # elif isinstance(key, type) and issubclass(key, Touching):
                                #     raise "key is a subclass of Touching"
                                # elif str(key) == "<class 'igibson.object_states.touching.Touching'>":
                                #     raise "str(key) == '<class 'igibson.object_states.touching.Touching'>'"
                                # elif key.__module__ == 'igibson.object_states.touching':
                                #     raise "key.__module__ == 'igibson.object_states.touching'"
                                # elif key.__name__ == 'Touching':
                                #     raise "key.__name__ == 'Touching'"
                                # elif "Touching" in str(key):
                                #     raise("Touching in str(key)")
                                # elif "Inside" in str(key):
                                #     raise("Inside in str(key)")
                                # else:
                                #     print(key)
                                #     print(type(key))
                                #     print(str(key))
                # except:
                #     # if this is not a binary state
                #     pass
    
    # return [int(row[1]) for row in data], contact_data
    return data, contact_data

In [6]:
def diff(list1, list2):
    # Ensure both lists are of the same length
    if len(list1) != len(list2):
        raise ValueError("Both lists must have the same length")
    
    # Count the number of differing entries
    differences = sum(1 for x, y in zip(list1, list2) if x != y)
    
    return differences

In [7]:
def main():
    args = Args(
    vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\vr\bottling_fruit_0_Wainscott_0_int_0_2021-05-24_19-46-46.hdf5",
    frame_save_path=r"C:\Users\Bryan\Documents\Code\demo\saved_frames\bottling_fruit",
    mode="simple"
)
    bddl.set_backend("iGibson")
    scene_graph_dict, contact_data_dict = replay_demo(
        args.vr_log_path,
        out_log_path=args.vr_replay_log_path,
        disable_save=args.disable_save,
        frame_save_path=args.frame_save_path,
        mode=args.mode,
        profile=args.profile,
    )
    
    return scene_graph_dict, contact_data_dict

# Replay Function

In [8]:
def replay_demo(
    in_log_path,
    out_log_path=None,
    disable_save=False,
    frame_save_path=None,
    verbose=True,
    mode="headless",
    start_callbacks=[],
    step_callbacks=[],
    end_callbacks=[],
    profile=False,
    image_size=(1280, 720),
):
    """
    Replay a BEHAVIOR demo.

    Note that this returns, but does not check for determinism. Use safe_replay_demo to assert for determinism
    when using in scenarios where determinism is important.

    @param in_log_path: the path of the BEHAVIOR demo log to replay.
    @param out_log_path: the path of the new BEHAVIOR demo log to save from the replay.
    @param frame_save_path: the path to save frame images to. None to disable frame image saving.
    @param mode: which rendering mode ("headless", "simple", "vr"). In simple mode, the demo will be replayed with simple robot view.
    @param disable_save: Whether saving the replay as a BEHAVIOR demo log should be disabled.
    @param profile: Whether the replay should be profiled, with profiler output to stdout.
    @param start_callback: A callback function that will be called immediately before starting to replay steps. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param step_callback: A callback function that will be called immediately following each replayed step. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param end_callback: A callback function that will be called when replay has finished. Should take a single
        argument, an iGBEHAVIORActivityInstance.
    @param image_size: The image size that should be used by the renderer.
    @return if disable_save is True, returns None. Otherwise, returns a boolean indicating if replay was deterministic.
    """
    # HDR files for PBR rendering
    hdr_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_02.hdr")
    hdr_texture2 = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_03.hdr")
    light_modulation_map_filename = os.path.join(
        igibson.ig_dataset_path, "scenes", "Rs_int", "layout", "floor_lighttype_0.png"
    )
    background_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "urban_street_01.jpg")

    # VR rendering settings
    vr_rendering_settings = MeshRendererSettings(
        optimized=True,
        fullscreen=False,
        env_texture_filename=hdr_texture,
        env_texture_filename2=hdr_texture2,
        env_texture_filename3=background_texture,
        light_modulation_map_filename=light_modulation_map_filename,
        enable_shadow=True,
        enable_pbr=True,
        msaa=False,
        light_dimming_factor=1.0,
    )

    # Check mode
    assert mode in ["headless", "vr", "simple", "pbgui"]

    # Initialize settings to save action replay frames
    vr_settings = VrSettings(config_str=IGLogReader.read_metadata_attr(in_log_path, "/metadata/vr_settings"))
    vr_settings.set_frame_save_path(frame_save_path)

    task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/atus_activity")
    if task is None:
        task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_name")
    task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/activity_definition")
    if task_id is None:
        task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_instance")
    scene = IGLogReader.read_metadata_attr(in_log_path, "/metadata/scene_id")
    physics_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/physics_timestep")
    render_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/render_timestep")
    filter_objects = IGLogReader.read_metadata_attr(in_log_path, "/metadata/filter_objects")
    instance_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/instance_id")
    urdf_file = IGLogReader.read_metadata_attr(in_log_path, "/metadata/urdf_file")
    
    

    if urdf_file is None:
        urdf_file = "{}_task_{}_{}_0_fixed_furniture".format(scene, task, task_id)

    if instance_id is None:
        instance_id = 0

    logged_git_info = IGLogReader.read_metadata_attr(in_log_path, "/metadata/git_info")
    logged_git_info = parse_str_config(logged_git_info)
    git_info = project_git_info()
    pp = pprint.PrettyPrinter(indent=4)

    for key in logged_git_info.keys():
        if key not in git_info:
            print(
                "Warning: {} not present in current git info. It might be installed through PyPI, "
                "so its version cannot be validated.".format(key)
            )
            continue

        logged_git_info[key].pop("directory", None)
        git_info[key].pop("directory", None)
        if logged_git_info[key] != git_info[key] and verbose:
            print("Warning, difference in git commits for repo: {}. This may impact deterministic replay".format(key))
            print("Logged git info:\n")
            pp.pprint(logged_git_info[key])
            print("Current git info:\n")
            pp.pprint(git_info[key])

    # VR system settings
    s = Simulator(
        mode=mode,
        physics_timestep=physics_timestep,
        render_timestep=render_timestep,
        rendering_settings=vr_rendering_settings,
        vr_settings=vr_settings,
        image_width=image_size[0],
        image_height=image_size[1],
    )

    igbhvr_act_inst = iGBEHAVIORActivityInstance(task, task_id)
    igbhvr_act_inst.initialize_simulator(
        simulator=s,
        scene_id=scene,
        scene_kwargs={
            "urdf_file": urdf_file,
        },
        load_clutter=True,
        online_sampling=False,
    )
    vr_agent = igbhvr_act_inst.simulator.robots[0]
    log_reader = IGLogReader(in_log_path, log_status=False)

    log_writer = None
    if not disable_save:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        if out_log_path == None:
            out_log_path = "{}_{}_{}_{}_{}_replay.hdf5".format(task, task_id, scene, instance_id, timestamp)

        log_writer = IGLogWriter(
            s,
            log_filepath=out_log_path,
            task=igbhvr_act_inst,
            store_vr=False,
            vr_robot=vr_agent,
            profiling_mode=profile,
            filter_objects=filter_objects,
        )
        log_writer.set_up_data_storage()

    #-----------------------------------------------------------------------------------------------
    # initiate frame count at -1 so the first frame is frame 0
    frame_count = -1
    scene_graph_dict = {}
    contact_data_dict = {}
    
    try:
        for callback in start_callbacks:
            callback(igbhvr_act_inst, log_reader)

        task_done = False
        while log_reader.get_data_left_to_read():
            

            igbhvr_act_inst.simulator.step(print_stats=profile)
            task_done |= igbhvr_act_inst.check_success()[0]
            
            # get the frame count and store the scene graph
            frame_count += 1
            scene_graph_dict[frame_count], contact_data_dict[frame_count] = get_flattened_scene_graph(igbhvr_act_inst)
            

            # Set camera each frame
            if mode == "vr":
                log_reader.set_replay_camera(s)

            for callback in step_callbacks:
                callback(igbhvr_act_inst, log_reader)

            # Get relevant VR action data and update VR agent
            vr_agent.apply_action(log_reader.get_agent_action("vr_robot"))

            if not disable_save:
                log_writer.process_frame()

        
        print("Demo was succesfully completed: ", task_done)
    
        demo_statistics = {}
        for callback in end_callbacks:
            callback(igbhvr_act_inst, log_reader)
    finally:
        s.disconnect()
        if not disable_save:
            log_writer.end_log_session()

    is_deterministic = None
    if not disable_save:
        is_deterministic = verify_determinism(in_log_path, out_log_path)
        print("Demo was deterministic: ", is_deterministic)

    demo_statistics = {
        "deterministic": is_deterministic,
        "task": task,
        "task_id": int(task_id),
        "scene": scene,
        "task_done": task_done,
        "total_frame_num": log_reader.total_frame_num,
    }
    
    print("Total frames counted by frame_counter: ", frame_count)
    #-----------------------------------------------------------------------------------------------

    
    
    return scene_graph_dict, contact_data_dict

# Config

In [9]:
# Manually specify the arguments here
args = Args(
    vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\vr\bottling_fruit_0_Wainscott_0_int_0_2021-05-24_19-46-46.hdf5",
    # frame_save_path=r"C:\Users\Bryan\Documents\Code\demo\saved_frames\bottling_fruit",
    mode="simple"
)

In [20]:
# Manually specify the arguments here
# args = Args(
#     vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\vr\filling_an_Easter_basket_0_Benevolence_1_int_1_2021-09-10_00-09-54.hdf5",
#     mode="simple"
# )

# Execute

In [9]:
scene_graphs, contact_data = main()

Warning, difference in git commits for repo: iGibson. This may impact deterministic replay
Logged git info:

{   'branch_name': 'behavior',
    'code_diff': 'diff --git a/gibson2/vr_config.yaml '
                 'b/gibson2/vr_config.yaml\n'
                 'index b6051117..57e3df2e 100644\n'
                 '--- a/gibson2/vr_config.yaml\n'
                 '+++ b/gibson2/vr_config.yaml\n'
                 '@@ -36,7 +36,7 @@ shared_settings:\n'
                 '   # Serial number of VR torso tracker - this can be found '
                 'by connecting/pairing the tracker,\n'
                 '   # then going into Steam VR settings -> controllers -> '
                 'manage vive trackers\n'
                 '   # Note: replace this with your own tracker serial number '
                 'or leave blank to not use one\n'
                 '-  torso_tracker_serial: "LHR-DF82C682"\n'
                 '+  torso_tracker_serial: "LHR-4F5EC31F"\n'
                 ' # Settings that are spe

INFO:root:IndoorScene model: Wainscott_0_int
INFO:root:StaticIndoorScene scene: Wainscott_0_int
INFO:root:Category walls
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\urdf\Wainscott_0_int_walls.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240925-164307_6971141076974386316_12988\walls_0.urdf
INFO:root:Category floors
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\urdf\Wainscott_0_int_floors.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240925-164307_6971141076974386316_12988\floor

indent from and: 1
indent from and: 1
SCENE NAME: Wainscott_0_int


INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240925-164307_6971141076974386316_12988\pot_plant_0_0.urdf
INFO:root:Category breakfast_table
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\objects\breakfast_table\5f3f97d6854426cfb41eedea248a6d25\5f3f97d6854426cfb41eedea248a6d25.urdf
INFO:root:Scale: [2.29448701 2.16354469 2.61762465]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240925-164307_6971141076974386316_12988\breakfast_table_1_0.urdf
INFO:root:Category straight_chair
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\objects\straight_chair\28a0b2a5afc96922ba63bc389be1ed5a\28a0b2a5afc96922

Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../../material/wall/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../../material/wall/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../../material/floor_1/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../../material/floor_1/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../../material/floor_0/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../../material/floor_0/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Wainscott_0_int\shape/visual\../..

# Save

In [13]:
with open(r"C:\Users\Bryan\Documents\Code\demo\segmentation\vector_scene_graphs\bottling_fruit.json", 'w') as json_file:
    json.dump(scene_graphs, json_file)

In [ ]:
with open(r"C:\Users\Bryan\Documents\Code\demo\segmentation\vector_scene_graphs\filling_an_Easter_basket.json", 'w') as json_file:
    json.dump(scene_graphs, json_file)

# Load Back

In [27]:
print(contact_data.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

# Process

In [18]:
contact_data

{0: [(('left_hand_1',
    igibson.object_states.touching.Touching,
    'strawberry_0_multiplexer'),
   False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'fridge_97'),
   False),
  (('left_hand_1',
    igibson.object_states.touching.Touching,
    'peach_0_multiplexer'),
   False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'countertop_84'),
   False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'jar_0'), False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'jar_1'), False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'carving_knife_0'),
   False),
  (('left_hand_1',
    igibson.object_states.touching.Touching,
    'bottom_cabinet_no_top_80'),
   False),
  (('left_hand_1',
    igibson.object_states.touching.Touching,
    'room_floor_kitchen_0'),
   False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'BRBody_1'),
   False),
  (('left_hand_1', igibson.object_states.touching.Touching, 'left_han

In [10]:
print(contact_data.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [12]:
print(scene_graphs.keys())


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [11]:
flattened_contact_data = {}
for key, value in contact_data.items():
    flattened_contact_data[key] = [int(row[1]) for row in value]

In [12]:
print(flattened_contact_data[0])

[]


In [13]:
common_dim = len(flattened_contact_data[0])
common_dim

0

In [14]:
diff_keys = []
diff_dims = []

for key, value in flattened_contact_data.items():
    if len(value) != common_dim:
        diff_keys.append(key)
        diff_dims.append(len(value))

print(diff_keys)
print(diff_dims)

[465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 503, 504, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846,

In [15]:
for i in diff_keys:
    print(i, contact_data[i])

465 [(('BRBody_1', 'right_hand_contact', 'fridge_97'), True), (('BRBody_1', 'right_hand_contact', 'fridge_97'), True), (('BRBody_1', 'right_hand_contact', 'fridge_97'), True), (('BRBody_1', 'right_hand_contact', 'fridge_97'), True), (('BRBody_1', 'right_hand_contact', 'fridge_97'), True)]
466 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
467 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
468 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
469 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
470 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
471 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
472 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
473 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
474 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
475 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
476 [(('BRBody_1', 'right_hand_grasped', 'fridge_97'), True)]
503 [(('BRBody_1', 'right_ha

In [39]:
contact_data[1831]

[(('strawberry_0_multiplexer',
   igibson.object_states.on_floor.OnFloor,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.on_floor.OnFloor,
   'right_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.touching.Touching,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.touching.Touching,
   'right_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.under.Under,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.under.Under,
   'right_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.next_to.NextTo,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.next_to.NextTo,
   'right_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.inside.Inside,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.inside.Inside,
   'right_han

In [38]:
contact_data[1832]

[(('strawberry_0_multiplexer',
   igibson.object_states.on_floor.OnFloor,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.on_floor.OnFloor,
   'right_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.touching.Touching,
   'left_hand_1'),
  False),
 (('strawberry_0_multiplexer',
   igibson.object_states.touching.Touching,
   'right_hand_1'),
  False),
 (('fridge_97', igibson.object_states.on_floor.OnFloor, 'left_hand_1'), False),
 (('fridge_97', igibson.object_states.on_floor.OnFloor, 'right_hand_1'),
  False),
 (('peach_0_multiplexer',
   igibson.object_states.on_floor.OnFloor,
   'left_hand_1'),
  False),
 (('peach_0_multiplexer',
   igibson.object_states.on_floor.OnFloor,
   'right_hand_1'),
  False),
 (('countertop_84', igibson.object_states.on_floor.OnFloor, 'left_hand_1'),
  False),
 (('countertop_84', igibson.object_states.on_floor.OnFloor, 'right_hand_1'),
  False),
 (('jar_0', igibson.object_states.on_floor.OnFloor,

In [33]:
contact_data[1950]

[]

In [37]:
CONTACT_THRESH = 1

key_frames = []
current_scene_contact = flattened_contact_data[0]

for key, value in flattened_contact_data.items():
    if key > 0 and len(value) == 10:
        similarity = diff(value, current_scene_contact)
        if similarity >= CONTACT_THRESH:
            if len(key_frames) == 0:
                key_frames.append(key)
                current_scene_contact = value
            elif key - key_frames[-1] > 3:
                key_frames.append(key)
                current_scene_contact = value
        else:
            pass
        

print(key_frames)

print(len(key_frames))

[]
0


In [44]:
STATE_THRESH = 10

In [53]:
key_frames = []
current_scene_graph = scene_graphs[0]

for key, value in scene_graphs.items():
    if key > 0:
        similarity = diff(value, current_scene_graph)
        if similarity >= STATE_THRESH:
            if len(key_frames) == 0:
                key_frames.append(key)
                current_scene_graph = value
            elif key - key_frames[-1] > 3:
                key_frames.append(key)
                current_scene_graph = value
        else:
            pass
        

print(key_frames)

print(len(key_frames))

[229, 954, 1033, 1224, 1539, 1697, 1832, 1836, 1841, 1867, 1871, 1899, 1903, 1941, 2114, 2118, 2122, 2126, 2137, 2144, 2148, 2153, 2237, 2277, 2284, 2348, 2367, 2377, 2383, 2410, 2425, 2429, 2433, 2549, 2564, 2568, 2576, 2599, 2603, 2608, 2612, 2618, 2624, 2636, 2640, 2649, 2653, 2657, 2661, 2754, 2759]
51


In [24]:
def format_filename(number):
    if not 0 <= number <= 9999:
        raise ValueError("Input must be between 0 and 9999")
    return f"{number:05d}.jpg"

format_filename(5)

'00005.jpg'

In [67]:
import os
import shutil

def cp_key_frames(key_frames, input_path, output_path):
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    for frame_number in key_frames:
        file_name =  f"{frame_number:05d}.jpg"
        input_file = os.path.join(input_path, file_name)
        output_file = os.path.join(output_path, file_name)
        
        if os.path.exists(input_file):
            shutil.copy2(input_file, output_file)
        else:
            print(f"Warning: {file_name} not found in {input_path}")

In [50]:
cp_key_frames(key_frames, r"C:\Users\Bryan\Documents\Code\demo\saved_frames\bottling_fruit", r"C:\Users\Bryan\Documents\Code\demo\saved_frames\test")

Copied 00229.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 00954.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01033.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01224.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01539.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01697.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01832.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01833.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01834.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01835.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01841.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01867.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01869.jpg to C:\Users\Bryan\Documents\Code\demo\saved_frames\test
Copied 01899.jpg to C:\Users\Bryan\Documents\Code\d

In [59]:
def create_folder_name(SIM_THRESH, TEMP_THRESH):
    # Convert the thresholds to strings
    sim_str = f"{SIM_THRESH:.2f}".rstrip('0').rstrip('.')
    temp_str = str(TEMP_THRESH)
    
    # Create the folder name with the desired format
    folder_name = f"SIM_{sim_str}_TEMP_{temp_str}"
    
    # Replace any remaining dots with underscores
    folder_name = folder_name.replace('.', '_')
    
    return folder_name

In [79]:
def segment_scene_graphs(SIM_THRESH=0.7, TEMP_THRESH=3, scene_graph_path=None, frame_path=None, output_path=None):
    """Takes in a dictionary of scene graphs and segments them based on cosine similarity"""
    
    
    out_folder = os.path.join(output_path, create_folder_name(SIM_THRESH, TEMP_THRESH))
    os.makedirs(out_folder, exist_ok=True)
    
    
    key_frames = []
    current_scene_graph = scene_graphs[0]

    for key, value in scene_graphs.items():
        if key > 0:
            similarity = cosine_similarity(value, current_scene_graph)
            if similarity < SIM_THRESH:
                if len(key_frames) == 0:
                    key_frames.append(key)
                    current_scene_graph = value
                elif key - key_frames[-1] > TEMP_THRESH:
                    key_frames.append(key)
                    current_scene_graph = value
            else:
                pass
            
    cp_key_frames(key_frames, frame_path, out_folder)
    
    print(f"The number of key frames selected is: {len(key_frames)}")
    

In [82]:
task_name = "bottling_fruit"

segment_scene_graphs(
    SIM_THRESH=0.5, 
    TEMP_THRESH=5, 
    scene_graph_path=rf"C:\Users\Bryan\Documents\Code\demo\segmentation\vector_scene_graphs\{task_name}.json", 
    frame_path=rf"C:\Users\Bryan\Documents\Code\demo\segmentation\demo_frames\{task_name}", 
    output_path=rf"C:\Users\Bryan\Documents\Code\demo\segmentation\state_segmentation\{task_name}",
)

The number of key frames selected is: 30
